# <font color=darkblue> Machine Learning model deployment with Flask framework</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with the help of the flask framework.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


### 2. Load the dataset

In [ ]:
import pandas as pd

# Load the dataset
file_path = "path_to_your_dataset_file.csv"  
car_data = pd.read_csv(file_path)

print(car_data.head())


### 3. Check the shape and basic information of the dataset.

In [ ]:

print("Shape of the dataset:", car_data.shape)

print("\nBasic Information about the Dataset:")
print(car_data.info())


### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [ ]:

duplicate_rows = car_data.duplicated()


num_duplicates = duplicate_rows.sum()


print("Number of duplicate rows:", num_duplicates)

car_data = car_data.drop_duplicates()


print("\nShape of the dataset after dropping duplicates:", car_data.shape)


### 5. Drop the columns which you think redundant for the analysis.

In [ ]:

redundant_columns = ["Car_Name"]
car_data = car_data.drop(columns=redundant_columns, axis=1)


print("Updated dataset after dropping redundant columns:")
print(car_data.head())


### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [ ]:

from datetime import date

current_year = date.today().year


car_data['age_of_the_car'] = current_year - car_data['Year']

car_data = car_data.drop(columns=['Year'], axis=1)

print("Updated dataset with 'age_of_the_car' feature:")
print(car_data.head())


### 7. Encode the categorical columns

In [ ]:

car_data_encoded = pd.get_dummies(car_data, columns=['Fuel_Type', 'Seller_Type', 'Transmission', 'Owner'], drop_first=True)


print("Updated dataset after one-hot encoding:")
print(car_data_encoded.head())


### 8. Separate the target and independent features.

In [ ]:

X = car_data_encoded.drop('Selling_Price', axis=1)
y = car_data_encoded['Selling_Price']  

print("Shape of independent features (X):", X.shape)
print("Shape of target variable (y):", y.shape)


### 9. Split the data into train and test.

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


### 11. Create a pickle file with an extension as .pkl

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


rf_model = RandomForestRegressor(random_state=42)

rf_model.fit(X_train, y_train)
\
train_predictions = rf_model.predict(X_train)
test_predictions = rf_model.predict(X_test)

train_r2_score = r2_score(y_train, train_predictions)
test_r2_score = r2_score(y_test, test_predictions)

print("R^2 Score - Training Set:", train_r2_score)
print("R^2 Score - Testing Set:", test_r2_score)


### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Car Price Prediction</title>
</head>
<body>
    <h2>Car Price Prediction</h2>
    
    <form action="/predict" method="post">
        <label for="year">Year of Purchase:</label>
        <input type="text" id="year" name="year" required><br>

        <label for="present_price">Present Price (in lakhs):</label>
        <input type="text" id="present_price" name="present_price" required><br>

        <label for="kms_driven">Kilometers Driven:</label>
        <input type="text" id="kms_driven" name="kms_driven" required><br>

        <label for="fuel_type">Fuel Type:</label>
        <select id="fuel_type" name="fuel_type" required>
            <option value="Petrol">Petrol</option>
            <option value="Diesel">Diesel</option>
            <option value="CNG">CNG</option>
        </select><br>

        <label for="seller_type">Seller Type:</label>
        <select id="seller_type" name="seller_type" required>
            <option value="Dealer">Dealer</option>
            <option value="Individual">Individual</option>
        </select><br>

        <label for="transmission">Transmission:</label>
        <select id="transmission" name="transmission" required>
            <option value="Manual">Manual</option>
            <option value="Automatic">Automatic</option>
        </select><br>

        <label for="owner">Owner:</label>
        <select id="owner" name="owner" required>
            <option value="First">First</option>
            <option value="Second">Second</option>
            <option value="Third">Third</option>
        </select><br>

        <input type="submit" value="Predict Price">
    </form>
</body>
</html>


### b) Create app.py file and write the predict function

In [ ]:
from flask import Flask, render_template, request
import pickle

app = Flask(__name__)


with open("model.pkl", "rb") as model_file:
    model = pickle.load(model_file)

@app.route('/')
def home():
    return render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():

    year = int(request.form['year'])
    present_price = float(request.form['present_price'])
    kms_driven = int(request.form['kms_driven'])
    fuel_type = request.form['fuel_type']
    seller_type = request.form['seller_type']
    transmission = request.form['transmission']
    owner = request.form['owner']

    fuel_type_encoded = 1 if fuel_type == 'Petrol' else (2 if fuel_type == 'Diesel' else 3)
    seller_type_encoded = 1 if seller_type == 'Dealer' else 0
    transmission_encoded = 1 if transmission == 'Manual' else 0
    owner_encoded = 1 if owner == 'First' else (2 if owner == 'Second' else 3)

  
    current_year = 2024  
    age_of_the_car = current_year - year


    prediction = model.predict([[age_of_the_car, present_price, kms_driven, fuel_type_encoded, 
                                 seller_type_encoded, transmission_encoded, owner_encoded]])

    return render_template('index.html', prediction_text=f'Predicted Price: {prediction[0]:.2f} Lakhs')

if __name__ == "__main__":
    app.run(debug=True)


### 13. Run the app.py python file which will render to index html page then enter the input values and get the prediction.

### Happy Learning :)